# **Medical Diagnoser: Predicting Diseases and Recommending Treatments**

### **Importing Libraries**

In [18]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [38]:
!pip install gradio

In [39]:
!pip install -U gradio

### **Loading the Dataset**

In [23]:
data = pd.read_csv('https://raw.githubusercontent.com/adil200/Medical-Diagnoser/main/medical_data.csv')
data.head()


,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


### **EDA of Data Features**

In [24]:
data['Disease']

0         Chronic Fatigue Syndrome
1               Migraine with Aura
2                   Hypothyroidism
3                    Mononucleosis
4                Diabetes Mellitus
                  ...             
402          Androgenetic Alopecia
403                 Conjunctivitis
404                         Stroke
405    Eustachian Tube Dysfunction
406              Colorectal Cancer
Name: Disease, Length: 407, dtype: object

In [46]:
data['Patient_Problem']

0      Constant fatigue and muscle weakness, struggli...
1      Frequent severe migraines, sensitivity to ligh...
2      Sudden weight gain and feeling cold, especiall...
3      High fever, sore throat, and swollen lymph nod...
4      Excessive thirst and frequent urination, dry m...
                             ...                        
402    Noticeable thinning of the hair on the top of ...
403    Greenish discharge from the eyes, accompanied ...
404    Experiencing confusion, difficulty speaking, a...
405    Constant feeling of fullness in the ears and h...
406    Blood in stool, along with a change in bowel m...
Name: Patient_Problem, Length: 407, dtype: object

In [47]:
data['Prescription']

0      Cognitive behavioral therapy, graded exercise ...
1      Prescription triptans, avoid triggers like bri...
2      Levothyroxine to regulate thyroid hormone levels.
3                Rest and hydration, ibuprofen for pain.
4                 Insulin therapy and lifestyle changes.
                             ...                        
402                    Minoxidil, finasteride for males.
403               Antibiotic or antihistamine eye drops.
404     Immediate medical attention, clot-busting drugs.
405             Nasal steroids, autoinflation exercises.
406      Colonoscopy, potentially surgery, chemotherapy.
Name: Prescription, Length: 407, dtype: object

### **Tokenizing and Sequencing Text Data**

In [25]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])


### **Padding Sequences**

In [26]:
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


### **Encoding the Labels and Converting them to Categorical**

In [27]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)


### **Combining Labels into a Multi-label Target Variable**

In [28]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))


### **Defining Model Architecture**

In [29]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax',
name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_),
activation='softmax', name='prescription_output')(lstm_layer)


### **Compiling the model**

In [30]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy',
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 17)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 17, 64)               320000    ['input_5[0][0]']             
                                                                                                  
 lstm_4 (LSTM)               (None, 64)                   33024     ['embedding_4[0][0]']         
                                                                                                  
 disease_output (Dense)      (None, 178)                  11570     ['lstm_4[0][0]']              
                                                                                            

### **Training the model**

In [31]:
model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output':
      prescription_labels_categorical}, epochs=200, batch_size=32) ##100 EPOCHS!!

Epoch 1/200
13/13 [==============================] - 4s 31ms/step - loss: 11.1448 - disease_output_loss: 5.1792 - prescription_output_loss: 5.9656 - disease_output_accuracy: 0.0074 - prescription_output_accuracy: 0.0000e+00
Epoch 2/200
13/13 [==============================] - 0s 30ms/step - loss: 11.1237 - disease_output_loss: 5.1610 - prescription_output_loss: 5.9628 - disease_output_accuracy: 0.0246 - prescription_output_accuracy: 0.0025
Epoch 3/200
13/13 [==============================] - 0s 31ms/step - loss: 11.0722 - disease_output_loss: 5.1032 - prescription_output_loss: 5.9690 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0025
Epoch 4/200
13/13 [==============================] - 0s 28ms/step - loss: 10.9706 - disease_output_loss: 4.9638 - prescription_output_loss: 6.0069 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0147
Epoch 5/200
13/13 [==============================] - 0s 18ms/step - loss: 10.8361 - disease_output_loss: 4.8848 - pr

### **Making Predictions**

In [45]:
def make_prediction(patient_problem):
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    prediction = model.predict(padded_sequence)

    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")


patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
#patient_input = "I've been experiencing frequent urination with a burning sensation"
#patient_input = "I've been feeling very down and unmotivated lately, and I have trouble sleeping"#
#patient_input = "I've been experiencing sharp chest pain on my left side, especially when I take a deep breath"
#patient_input = "I feel nauseous and have been vomiting several times in the past few hours"
#patient_input = "I've been experiencing sharp chest pain on my left side, especially when I take a deep breath"
make_prediction(patient_input)


1/1 [==============================] - 0s 35ms/step
Predicted Disease: Depression
Suggested Prescription: Antidepressants; eating nutrient-rich foods.


### **Model Evaluation**

In [36]:
def evaluate_model():
    y_pred = model.predict(padded_sequences)
    y_disease_pred = np.argmax(y_pred[0], axis=1)
    y_prescription_pred = np.argmax(y_pred[1], axis=1)

    #print("Classification Report for Disease:")
    #print(classification_report(disease_labels, y_disease_pred, zero_division=0))

    print("Confusion Matrix for Disease:")
    print(confusion_matrix(disease_labels, y_disease_pred))

    #print("Classification Report for Prescription:")
    #print(classification_report(prescription_labels, y_prescription_pred, zero_division=0))

    print("\nConfusion Matrix for Prescription:")
    print(confusion_matrix(prescription_labels, y_prescription_pred))

    # Calculate and print accuracy
    accuracy_disease = accuracy_score(disease_labels, y_disease_pred)
    accuracy_prescription = accuracy_score(prescription_labels, y_prescription_pred)
    print(f"\nAccuracy for Disease: {accuracy_disease}")
    print(f"Accuracy for Prescription: {accuracy_prescription}")

# Run the evaluation
evaluate_model()

13/13 [==============================] - 0s 9ms/step
Confusion Matrix for Disease:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 2]]

Confusion Matrix for Prescription:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]

Accuracy for Disease: 0.9336609336609336
Accuracy for Prescription: 0.7567567567567568


### **Gradio Interface for Model Prediction**

In [41]:
import gradio as gr

def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Making prediction
    prediction = model.predict(padded_sequence)

    # Decoding the prediction
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    return disease_predicted, prescription_predicted

iface = gr.Interface(
    fn=make_prediction,
    inputs=gr.Textbox(label="Enter Patient's Problem"),
    outputs=["text", "text"],
    title="Medical Diagnoser",
    description="Enter the patient's problem and get the predicted disease and suggested prescription."
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2bc105434d0f51dcdf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### **Patient Problem Prompts for Input**

Chest Pain: "I've been experiencing sharp chest pain on my left side, especially when I take a deep breath."

Skin Rash: "I have a red, itchy rash that has appeared on my arms and legs over the past few days."

Fever and Chills: "I've had a high fever and chills for the past two days, along with feeling very tired."

Shortness of Breath: "I'm having trouble catching my breath, even when I'm sitting down and resting."

Headache and Dizziness: "I've had a severe headache for the past few hours, and I feel dizzy when I stand up."

Joint Pain and Stiffness: "My joints have been feeling stiff and painful, especially in the mornings."

Urinary Tract Infection (UTI) Symptoms: "I've been experiencing frequent urination with a burning sensation."

Stomach Ache and Diarrhea: "I have a bad stomach ache and have been experiencing frequent diarrhea."

Vision Problems: "My vision has become blurry in my right eye over the past few days."

Mental Health: "I've been feeling very down and unmotivated lately, and I have trouble sleeping."


**Respiratory:**

Cough: "I've had a persistent cough for the past week, along with wheezing and difficulty breathing."

Sore Throat: "My throat is very sore and painful, especially when I swallow. I also have a headache and feel tired."

Sinus Congestion: "I have a stuffy nose and congestion in my sinuses. My face also feels pressure."

**Cardiovascular:**


Palpitations: "My heart is racing and pounding very fast, even when I'm at rest."

Chest Tightness: "I feel a tightness or pressure in my chest, especially when I exert myself."

Swollen Ankles: "My ankles are swollen and feel heavy, particularly at the end of the day."


**Digestive:**

Nausea and Vomiting: "I feel nauseous and have been vomiting several times in the past few hours."

Heartburn: "I experience a burning sensation in my chest after eating, especially spicy or fatty foods."

Constipation: "I haven't had a bowel movement in several days and feel bloated and uncomfortable."


**Neurological:**

Numbness and Tingling: "My hands and feet are feeling numb and tingly, and I have a pins-and-needles sensation."

Seizure: "I experienced a sudden episode of uncontrolled muscle jerking and loss of consciousness."

Weakness: "I feel weak and have difficulty lifting objects or climbing stairs."
Endocrine:

Excessive Thirst: "I'm constantly thirsty and drink large amounts of fluids, but I still feel dehydrated."

Fatigue: "I'm feeling extremely tired and lack energy, even after a good night's sleep."

Weight Changes: "I've experienced unexplained weight gain (or loss) recently, despite no changes in my diet or exercise."


**Allergic:**

Food Allergies: "After eating certain foods, I experience hives, swelling, and difficulty breathing."

Seasonal Allergies: "My eyes are itchy and watery, and I have a runny nose during specific seasons."

Medication Allergies: "I had a severe reaction (rash, swelling, etc.) after taking a new medication."


**Mental Health:**

Anxiety: "I feel anxious and on edge most of the time, with racing thoughts and difficulty relaxing."

Panic Attack: "I experience sudden episodes of intense fear with shortness of breath, dizziness, and chest pain."